# Raspberrypi4 Install

Follow the steps in order to properly install the Raspberrypi:

- Install Raspberry Pi OS using Raspberry Pi Imager (download for windows): https://www.raspberrypi.org/software/
- Run the application and save the image (Raspberry pi OS (32-bit)) to the SD card
- Insert the SD in a RBPi board and connect it to screen
- The system ask you to choose a wifi network and change the timezone, language and password
- Reboot and select Raspberry Pi Configuration to activate the SSH and Camera
- reboot and open a terminal to obtain the IP address using "ifconfig"

You are now ready to connect from your Ubuntu Virtual machine using VS Code or a simple terminal

<img src="./Images/5_rbpi4_imager.png">

Alternativelly you can connect to the RBPi using SSH in a wifi network without the previous configurations using a screen connection if you follow the instructions:

- add ssh file (without extension) in the SD in the G:boot directory
- add wpa_supplicant.conf file containing different wifi.

### Win IP Scanner
Install IP scanner for windows: https://www.advanced-ip-scanner.com/es/

Connect the RBPi to the router and find the IP number

### Connection to RBPi from Ubuntu PC
The first connection to the RBPi has to be using a Terminal in Virtual Box:

- type "ssh pi@192.168.18.46" (the default user:pi and pass:raspberry)
- When you connect the same RBPi to another network, you have a warning because you have to regenerate the KeyGen of your computer. You have to type (using the proper IP): ssh-keygen -R 192.168.18.46

### Remote Desktop VNC
This Desktop is available for Windows and MAC users.

- To begin, open up the raspi-config tool by running the following command: sudo raspi-config
- Within the configuration tool, use the arrow keys to scroll down to "2. Display Options" and "D1. Resolution"
- Select the desired resolution. Usually DMT Mode 82 1920x1080 60Hz 16:9
- Press OK
- Within the configuration tool, use the arrow keys to scroll down to "3. Interface Options" and "P1. Camera" and "P3. VNC"
- Select VNC enable
- Press "finish" and "OK" and reboot

Now in your PC install VNC Viewer: https://www.realvnc.com/es/connect/download/viewer/raspberrypi/

You will be able to enter in the raspberrypi4 desktop

### Install ROS Melodic

Follow the instructions on the document: ROS Melodic on Raspbian.pdf

Refer to the official webpage: http://wiki.ros.org/ROSberryPi/Installing%20ROS%20Melodic%20on%20the%20Raspberry%20Pi

In [ ]:
sudo sh -c 'echo "deb http://packages.ros.org/ros/ubuntu $(lsb_release -sc) main" > /etc/apt/sources.list.d/ros-latest.list'

In [ ]:
sudo apt-key adv --keyserver hkp://ha.pool.sks-keyservers.net:80 --recv-key C1CF6E31E6BADE8868B172B4F42ED6FBAB17C654

In [ ]:
sudo apt-get update

In [ ]:
sudo apt install -y python-rosdep python-rosinstall-generator python-wstool python-rosinstall build-essential cmake

In [ ]:
sudo rosdep init

In [ ]:
rosdep update

In [ ]:
mkdir -p ~/ros_catkin_ws

In [ ]:
cd ~/ros_catkin_ws

In [ ]:
rosinstall_generator desktop --rosdistro melodic --deps --wet-only --tar > melodic-desktop-wet.rosinstall

In [ ]:
wstool init -j4 src melodic-desktop-wet.rosinstall

Install the compatible version of Assimp (Open Asset Import Library) to fix collada_urdf dependency problem and OGRE for rviz.

In [ ]:
cd ~
mkdir -p ~/ros_catkin_ws/external_src
cd ~/ros_catkin_ws/external_src

download the file "assimp-3.1.1_no_test_models.zip"

In [ ]:
wget https://sourceforge.net/projects/assimp/files/assimp-3.1/assimp-3.1.1_no_test_models.zip

In [ ]:
unzip assimp-3.1.1_no_test_models.zip
cd assimp-3.1.1/
cmake .
make
sudo make install
sudo apt-get install libogre-1.9-dev

Resolving Dependencies

In [ ]:
cd ~/ros_catkin_ws/
rosdep install -y --from-paths src --ignore-src --rosdistro melodic -r --os=debian:buster

In [ ]:
sudo ./src/catkin/bin/catkin_make_isolated --install -DCMAKE_BUILD_TYPE=Release --install-space /opt/ros/melodic -j2

In [ ]:
echo "source /opt/ros/melodic/setup.bash" >> ~/.bashrc

In [ ]:
sudo reboot

In [ ]:
roscore

### Install Sensor packages
#### 1. Camera RaspberryPi

In [ ]:
git clone https://github.com/UbiquityRobotics/raspicam_node.git
git clone --single-branch --branch=indigo-devel https://github.com/ros-perception/image_transport_plugins.git
git clone --single-branch --branch=melodic https://github.com/ros-perception/vision_opencv.git
git clone --single-branch --branch=hydro-devel https://github.com/ros-perception/image_common.git

#### 2. RP-LIDAR

#### 3. YD-LIDAR

In [ ]:
git clone https://github.com/EAIBOT/ydlidar.git

Plug the LIDAR USB and we want pi and dialout as owner/owner group

In [ ]:
ls -la /dev/ttyUSB*
sudo chown pi:dialout /dev/ttyUSB0
roscd ydlidar/startup
sudo chmod a+rwx ./*
sudo sh initenv.sh

#### 4. gopigo3

In [ ]:
git clone https://github.com/ros-gopigo/gopigo3_node
git clone --single-branch --branch=kinetic-devel https://github.com/ros-teleop/teleop_tools.git

## Setup rUBot with sensors
We will execute in different terminals:

- the rubot_mecanum node for driving control
- the ydlidar node
- the raspicam node

In [ ]:
roslaunch mecanum_control rubot_control1.launch
roslaunch ydlidar lidar.launch
roslaunch raspicam_node camerav2_1280x960_10fps.launch enable_raw:=true camera_frame_id:="laser_frame"

<img src="./Images/5_rubot_nodes.png">